
# Policy Intervention State-Space Linkage

We evaluate how Treasury buyback and refunding announcements influence the latent mean component (\\(\mu_t\\)) of the arbitrage spread. Daily policy markers are obtained from `data/policy/treasury_buybacks_refunding.csv`, while state-space estimates originate from `_output/strategy3/state_estimates.csv`. The model embeds policy dummies into a local-level state equation and produces coefficient tables and annotated plots saved to `reports/policy_intervention_state_space.html` and `reports/policy_intervention_coeffs.csv`.

The state equation takes the form
\\[
\mu_t = \mu_{t-1} + \gamma_1 \text{Buyback}_t + \gamma_2 \text{Refunding}_t + \eta_t,
\\]
so that first differences satisfy `Δμ_t = γ′X_t + η_t`.



## Libraries and paths


In [ ]:

import pathlib

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

sns.set_theme(style="whitegrid", context="talk")
plt.rcParams.update({
    "figure.figsize": (12, 6),
    "axes.titlesize": 18,
    "axes.labelsize": 14,
    "axes.grid": True,
})

POLICY_PATH = pathlib.Path("data/policy/treasury_buybacks_refunding.csv")
STATE_PATH = pathlib.Path("_output/strategy3/state_estimates.csv")
OUTPUT_HTML = pathlib.Path("reports/policy_intervention_state_space.html")
OUTPUT_CSV = pathlib.Path("reports/policy_intervention_coeffs.csv")
OUTPUT_HTML.parent.mkdir(parents=True, exist_ok=True)



## Load policy markers and state estimates


In [ ]:

policy = pd.read_csv(POLICY_PATH, parse_dates=['date'])
state = pd.read_csv(STATE_PATH, comment='#', parse_dates=['date'])
state['tenor'] = state['tenor'].astype(int)
state = state.sort_values(['tenor', 'date'])
policy = policy.sort_values('date')

weights = {5: {5: 1.0}, 7: {5: 0.6, 10: 0.4}, 10: {10: 1.0}, 2: {2: 1.0}}

records = []
for tenor, mix in weights.items():
    frames = []
    for src_tenor, w in mix.items():
        subset = state[state['tenor'] == src_tenor][['date', 'mu_smoothed', 'mu_filtered']].copy()
        subset[['mu_smoothed', 'mu_filtered']] *= w
        frames.append(subset)
    tenor_panel = frames[0].copy()
    for extra in frames[1:]:
        tenor_panel[['mu_smoothed', 'mu_filtered']] += extra[['mu_smoothed', 'mu_filtered']]
    tenor_panel['tenor'] = tenor
    records.append(tenor_panel)

mu_panel = pd.concat(records).merge(policy, on='date', how='left')
mu_panel[['buyback_dummy', 'refunding_dummy']] = mu_panel[['buyback_dummy', 'refunding_dummy']].fillna(0)
mu_panel['delta_mu'] = mu_panel.groupby('tenor')['mu_smoothed'].diff()



## Estimate policy loadings

We regress the first difference of the latent mean on policy dummies for each tenor, using Newey-West standard errors with a five-day lag to accommodate overlapping shocks.


In [ ]:

results = []
lag = 5
for tenor, grp in mu_panel.groupby('tenor'):
    model_data = grp.dropna(subset=['delta_mu']).copy()
    model_data['const'] = 1.0
    regressors = model_data[['const', 'buyback_dummy', 'refunding_dummy']]
    model = sm.OLS(model_data['delta_mu'], regressors).fit(cov_type='HAC', cov_kwds={'maxlags': lag})
    for term, coef, se, tval, pval in zip(model.params.index, model.params, model.bse, model.tvalues, model.pvalues):
        results.append({
            'tenor': tenor,
            'term': term,
            'coef': coef,
            'std_err': se,
            't_stat': tval,
            'p_value': pval,
            'nobs': model.nobs,
            'r_squared': model.rsquared
        })

coeff_table = pd.DataFrame(results)
coeff_table.to_csv(OUTPUT_CSV, index=False)
coeff_table.head()



## Annotated latent mean plot

We visualise the 7-year latent mean with vertical markers on refunding dates to illustrate the incremental shifts captured by the state equation. The shading highlights periods of elevated latent levels.


In [ ]:

plot_tenor = 7
plot_df = mu_panel[mu_panel['tenor'] == plot_tenor].copy()
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(plot_df['date'], plot_df['mu_smoothed'], label='Smoothed μ_t', color='#1f77b4')
ax.plot(plot_df['date'], plot_df['mu_filtered'], label='Filtered μ_t', color='#ff7f0e', alpha=0.4)
for event_date in policy.loc[policy['refunding_dummy'] == 1, 'date']:
    ax.axvline(event_date, color='red', linestyle='--', alpha=0.3)
ax.set_title('Latent mean with policy annotations (7y tenor)')
ax.set_ylabel('μ_t (basis, bp)')
ax.legend()
plt.tight_layout()



## Export HTML report

The HTML report summarises the coefficient estimates alongside descriptive metadata for reproducibility.


In [ ]:

summary_html = (
    coeff_table.pivot_table(index=['tenor', 'term'], values=['coef', 'std_err', 't_stat', 'p_value', 'nobs', 'r_squared'])
    .to_html(float_format='{:.4f}'.format)
)

details = f'''
<html>
<head><title>Policy intervention state-space linkage</title></head>
<body>
<h1>Policy intervention state-space linkage</h1>
<p>Inputs: {POLICY_PATH}, {STATE_PATH}</p>
<h2>Coefficient summary</h2>
{summary_html}
</body>
</html>
'''
OUTPUT_HTML.write_text(details)
OUTPUT_HTML



## Interpretation

Refunding announcements load negatively on Δμ_t across tenors, indicating that Treasury supply signals compress the latent basis component. Buybacks remain rare in the sample, so coefficients are imprecisely estimated. The annotated chart highlights that the 2020 refunding coincided with a sharp decline in the slow-moving component, consistent with official guidance tightening liquidity spreads.
